# **Collaborative recommender module**

## **Introduction**

Collaborative recommender module is constructed to make personalized recommendations that are highly rated by users having similar user profiles.

Yelp dataset contains about 200K business, 2M users and 8M reviews, the user - restaurants interaction matrix would be a sparse matrix. Thus matrix factorization algorithms are used to construct the matrix and provide recommendations.

## **Implementation Strategy**

### 1 **Ranking**
#### 1.1 **SVD (Singular Value Decomposition)**
#### 1.1.1 **SVD without bias** 
The user - retaurants matrix is factorized into user latent features and restaurants latent features matrix using SVD algorithm.

#### 1.1.2 **SVD with bias**
To the original SVD matrices, user bias and restaurant bias matrix are introduced. 

`pred_rating = user latent features X restaurants latent features + user bias + restaurants bias + global_mean `

#### 1.2 **NMF (Non negative matrix factorization)**
#### 1.2.1 **NMF without bias**
The user - retaurants matrix is factorized into non negative user latent features and non negative restaurants latent features matrix using NMF algorithm.

#### 1.2.2 **NMF with bias**
To the original NMF matrices, user bias and restaurant bias matrix are introduced.

`scikit-suprise`, and `scikit-learn` packages are used fro prototyping the mentioned algorithms. Prototyping results indicates, amongst all four algorithms SVD with bias and NMF without bias provides acceptable ratings predictions. Being flexible, and best performing SVD with bias is used for module implementation. 

### 2 **Implementation**

#### 2.1 **Optimization**
Based on the prototyping results SVD with bias algorithm is optimized via gridsearch cross validation. 

#### 2.2 **Evaluation**
**RMSE** <br>
of the model are ----------- for testset with new users or restaurants, testset with no new users or restaurants, and testset with only users or restaurants with more than 5 ratings. 

**NDCG** <br>

#### 2.3 **Module Development**
1. All relevant restaurants are filtered for Collaborative Recommendations module
2. Extracted latent matrix, bias matrix for user and item from trained, optimized SVD algorithm are fed to class
3. For given user id, ratings are predicted for all filtered restaurants by multiplying user laten and item latent, adding biases for user, item as well as global mean ratings
4. Predicted ratings are paired with correspoding restaurants and filtered the list by unrated restaurants by user
5. Recommendations from this module are merged with content based recommender module's recommendations and after sorting the list by predicted ratings, final recommendations are displayed

#### 2.4 Testing
Different test cases are implemented to see completeness and computing time.

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle

# For loop visualization
from tqdm import tqdm

# For graphical representation
%matplotlib inline
import matplotlib.pyplot as plt

# For algorithm, optimization, evaluation
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, accuracy, NMF

In [3]:
business = pd.read_csv('clean_business.csv')
review = pd.read_csv('clean_review.csv')

In [4]:
print('Number of restaurants in business dataset: ', len(business.business_id.unique()))
print('Number of restaurants in review dataset: ', len(review.business_id.unique()))

set_bus = set(business.business_id.unique())
set_rev = set(review.business_id.unique())

if len(set_bus) == len(set_bus.intersection(set_rev)):
    print('\nAll business_id from business dataset can be found in review dataset')
else:
    print('\nNot all business_id from business dataset can be found in review dataset')

Number of restaurants in business dataset:  44202
Number of restaurants in review dataset:  209394

All business_id from business dataset can be found in review dataset


In [5]:
# Reduced review by removing the duplicated user, restaurant rating combinations

review_r = review[~review.duplicated(['user_id','business_id'], keep='first')]

# Keep only required columns
review_r = review_r[['user_id', 'business_id', 'stars']]
review_r = review_r.dropna(axis='index')
review_r.reset_index(inplace=True, drop=True)

print('Original datset length: ', len(review)) 
print('Reduced dataset length: ', len(review_r))

Original datset length:  8021124
Reduced dataset length:  7735089


In [6]:
review_r.head()

,user_id,business_id,stars
0,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0
1,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0
2,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0
3,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0
4,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0


### 1 **Ranking**
#### 1.1 **SVD (Singular Value Decomposition)**
#### 1.1.1 **SVD without bias**

In [7]:
%%time

# Reader object with rantings scale
reader = Reader(rating_scale=(1, 5))

# Load trainset, NOTE: the columns must correspond to user id, item id and ratings in the exact order
data = Dataset.load_from_df(review_r, reader)

# Build training, testing data
trainset, testset = train_test_split(data, test_size=0.20)

# Algorithm

algo = SVD(n_factors=20, n_epochs = 30, biased=False)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

# Gather useful info from model
mean = algo.trainset.global_mean 
user_latent, item_latent = algo.pu, algo.qi

RMSE: 1.6311
CPU times: user 7min 52s, sys: 7.57 s, total: 7min 59s
Wall time: 7min 59s


#### 1.1.2 **SVD with bias**

In [8]:
%%time

# Reader object with rantings scale
reader = Reader(rating_scale=(1, 5))

# Load trainset, NOTE: the columns must correspond to user id, item id and ratings in the exact order
data = Dataset.load_from_df(review_r, reader)

# Build training, testing data
trainset, testset = train_test_split(data, test_size=0.20)

# Algorithm

algo_bias = SVD(n_factors=20, n_epochs = 30, biased=True)
algo_bias.fit(trainset)
predictions_bias = algo_bias.test(testset)
accuracy.rmse(predictions_bias)

# Gather useful info from model
mean_bias = algo_bias.trainset.global_mean 
user_latent_bias, item_latent_bias = algo_bias.pu, algo_bias.qi
user_bias, item_bias = algo_bias.bu, algo_bias.bi

RMSE: 1.3132
CPU times: user 7min 58s, sys: 5.88 s, total: 8min 4s
Wall time: 8min 4s


#### 1.2 NMF (Non Negative Matrix Factorization)
#### 1.2.1 NMF without bias

In [9]:
%%time

algo_nmf = NMF(n_factors=20, n_epochs = 30, biased=False)
algo_nmf.fit(trainset)
predictions_nmf = algo_nmf.test(testset)
accuracy.rmse(predictions_nmf)

# Gather useful info from model
mean_nmf = algo_nmf.trainset.global_mean 
user_latent_nmf, item_latent_nmf = algo_nmf.pu, algo_nmf.qi

RMSE: 1.4638
CPU times: user 9min 44s, sys: 7.06 s, total: 9min 51s
Wall time: 9min 51s


#### 1.2.2 MNF with bias

In [10]:
%%time

algo_nmf_bias = NMF(n_factors=20, n_epochs = 30, biased=True)
algo_nmf_bias.fit(trainset)
predictions_nmf_bias = algo_nmf_bias.test(testset)
accuracy.rmse(predictions_nmf_bias)

# Gather useful info from model
mean_nmf_bias = algo_nmf_bias.trainset.global_mean 
user_latent_nmf_bias, item_latent_nmf_bias = algo_nmf_bias.pu, algo_nmf_bias.qi
user_nmf_bias, item_nmf_bias = algo_nmf_bias.bu, algo_nmf_bias.bi

RMSE: 1.9850
CPU times: user 9min 22s, sys: 5 s, total: 9min 27s
Wall time: 9min 27s


### 2 **Implementation**
#### 2.1 **Optimization**

#### **Optimized parameters = {'n_factors': 10, 'n_epochs': 50, 'lr_all': 0.005, 'biased': True}**
(ran this optimization task on seperate notebook and took 5 hours to run on 8 core CPU, 32 GB VM)

#### 2.2 **Evaluation**
#### **NDCG** <br>

In [11]:
def dcg_r(r, k):
    r = np.asfarray(r)[:min(len(r), k)]                                    # Convert to float type numpy array
    if r.size:
        return np.sum(r/np.log2(np.arange(2, r.size + 2)))
    return None

def ndcg_r(r, k):
    idcg = dcg_r(sorted(r, reverse=True), k)
    dcg = dcg_r(r, k)
    if idcg == None or dcg == None:
        return None
    return dcg/idcg

In [12]:
%%time

# Retrain with optimized parameters
algo_optimized = SVD(n_factors=10, n_epochs=50, biased=True)
algo_optimized.fit(trainset)
predictions = algo_optimized.test(testset)
accuracy.rmse(predictions)

# Gather useful info from model
mean_bias = algo_optimized.trainset.global_mean 
user_latent_bias, item_latent_bias = algo_optimized.pu, algo_optimized.qi
user_bias, item_bias = algo_optimized.bu, algo_optimized.bi

RMSE: 1.3392
CPU times: user 9min 18s, sys: 777 ms, total: 9min 19s
Wall time: 9min 19s


In [13]:
pred_one = algo_optimized.test(testset)
accuracy.rmse(pred_one)

RMSE: 1.3392


1.339154610894873

In [14]:
rating_predict_1 = pd.DataFrame(index=np.arange(len(pred_one)),columns=['user_id','business_id','stars','rating_predict'])

i=0
for entry in tqdm(pred_one):
    rating_predict_1.iloc[i,:] = entry.uid, entry.iid, entry.r_ui, entry.est
    i += 1    
assert i == len(pred_one)

100%|██████████| 1547018/1547018 [01:55<00:00, 13431.41it/s]


In [15]:
%%time

# Particular user_id as an example
user_id = '---1lKK3aKOuomHnwAkAow'                                              # 12 review ratings available in 'testset'

# Rank the 'rating_predict' dataframe by the predicted ratings in descending order
rating_predict_1 = rating_predict_1.sort_values('rating_predict', ascending=False)

# Filter to the user_id of interest only
rec = rating_predict_1[rating_predict_1.user_id == user_id].set_index('business_id')[['rating_predict','stars']]
print('Ranking by predicted ratings:\n', rec)

# NDCG @top 10 and @top 5
NDCG_10 = ndcg_r(r=rec.stars.values, k=10)
NDCG_5 = ndcg_r(r=rec.stars.values, k=5)

print('\nNormalized discounted cumulative gain achieved at top-10 based on testset:\n', NDCG_10)
print('\nNormalized discounted cumulative gain achieved at top-5 based on testset:\n', NDCG_5)

Ranking by predicted ratings:
                        rating_predict stars
business_id                                
t6WY1IrohUecqNjd9bG42Q              5   4.0
N8Rwk4XrKaHYXXninuxg9Q              5   5.0
zEaGcSVPDQipnRdEZp-F6g              5   2.0
Ks0M3J4vZAKsHPuCINz5fQ              5   4.0
AZlnpvILz5cEWJifjr2CSQ              5   5.0
mz9ltimeAIy2c2qf5ctljw              5   5.0
Gv7v-b3Fr_Gvrt6jvmpkJA       4.831041   5.0
Od2VpwoOBxycNyQNOMJ6eQ       4.624336   1.0
PSRZaGGxXmOmabL2si8pKw       4.519192   3.0
YCEZLECK9IToE8Mysorbhw       4.446617   5.0
edV_IqWqz5KVSGLrsru5EQ       4.420963   5.0
ebJC4iq6oUuiE_XBoEAAOg       4.386198   5.0
OicpDroqnfmbtw5jSgf4lQ       4.335283   5.0
CWNMLT-ppaUjLMmrnYDPVg       4.234165   5.0
cHuA0Yb5oYwx1lrNVABqdQ        4.09728   5.0
R-McIj4Psxl1VlEacsXeRg       4.096885   4.0
WOO81gScY3_VpaIfXFAKpw       4.016539   4.0
gJd1jOcl5FdqY020Q19n6Q       3.791256   5.0
DXlDzOcpdUE_F21tok0fgw       3.758748   4.0
x-80R4DE6AsYoDtA2H-sKw       3.742237   3.0
i

In [16]:
# Store useful information from trained model

useful_info = {'mean_rating': algo_optimized.trainset.global_mean,
               'user_latent': algo_optimized.pu,
               'item_latent': algo_optimized.qi,
               'user_bias': algo_optimized.bu,
               'item_bias': algo_optimized.bi,
               'userid_to_index': algo_optimized.trainset._raw2inner_id_users,
               'itemid_to_index': algo_optimized.trainset._raw2inner_id_items
              }

In [17]:
# Pickle useful info 
with open('svd_algo_trained_info', 'wb') as f:
    pickle.dump(useful_info, f)
    
# Pickle trained model
with open('svd_bias_algo_trained', 'wb') as f:
    pickle.dump(algo_optimized, f)

In [24]:
business = pd.read_csv('clean_business.csv')
review = pd.read_csv('clean_review.csv')

mean_global = ((business.stars * business.review_count).sum() / (business.review_count.sum()))
k = 30                                                                  # 50% quantile of the review counts 

business['stars_adj'] = ((business.review_count * business.stars) + (k * mean_global)) / (business.review_count + k)

In [25]:
class Recommender_Engine:
    
    def __init__(self, n=10, stars_original=False):
        """
        Instantiate the object. Default setting for ranking would be stars_adj with top 10 recommendations.
        """
        
        self.n = n                                                     # Number of recommendations
        self.stars_original = stars_original                           # Boolean for ranking method                            
        self.disply_columns = ['name', 'address', 'city','state',\
                               'attributes.RestaurantsPriceRange2',\
                               'review_count','stars','stars_adj',\
                               'cuisine','style']                    # List of columns to be displayed in the results
        
        if self.stars_original:
            score = 'stars'
        else:
            score = 'stars_adj'
            
        self.recommendation = business[business.is_open == 1].sort_values(score, ascending=False)
                                                                      # Filter only open restaurants
    
    def display(self):
        
        if len(self.recommendation) == 0:
            print("Sorry, there are no matching recommendations.")
        elif self.n < len(self.recommendation):
            print("Below is the list of the top {} recommended restaurants for you: ".format(self.n))
            print(self.recommendation.iloc[:self.n][self.disply_columns])
        else:
            print("Below is the list of the top {} recommended restaurants for you: ".format(len(self.recommendation)))
            print(self.recommendation.iloc[self.disply_columns]) 
    
            
    def collaborative_filtering(self, user_id=None):
        self.user_id = user_id
        if self.user_id is None:
            print('User ID is not provided')
        if len(user_id) != 22:                                        # Sanity check on length of user id
            print('Invalid user ID')
            return None
        
        self.recommendation = business[business.is_open == 1]
        if 'stars_pred' in self.recommendation.columns:
            self.recommendation.drop('stars_pred', axis=1, inplace=True)
            
        self.display_columns = ['name', 'address', 'city','state',\
                                'attributes.RestaurantsPriceRange2',\
                                'review_count','stars','stars_adj',\
                                'cuisine','style']
            
        with open('svd_algo_trained_info', rb) as f:
            useful_info = pickle.load(f)
            
        mean_rating = userful_info['mean_rating']
        user_latent = userful_info['user_latent']
        item_latent = userful_info['ietm_latent']
        user_bias = userful_info['user_bias']
        item_bias = userful_info['item_bias']
        userid_idx = userful_info['userid_to_index']
        itemid_idx = userful_info['itemid_to_index']
        
        # Recommendations
        if self.user_id in userid_idx:
            u_idx = userid_idx[self.user_id]
            pred = mean_rating + user_bias[u_idx] + item_bias + np.dot(user_latent[u_idx,:], ietm_latent.T)
        else:
            print('Sorry, no personlaized recommendations yet!')
            print('\nHere are generic recommendations: ')
            
            pred = mean_rating + ietm_bias
            
        prediction = pd.DataFrame(data=pred, index=itemid_idx.values(), columns=['stars_pred'])
        prediction.index.name == 'matrix_item'
        assert len(prediction) == len(pred)
        prediction['business_id'] = list(itemid_idx.keys())
        
        # Filter to unrated business by user
        if self.user_id in userid_idx:
            rated_bus = review[review.user_id == self.user_id].business_id.unique()
            prediction = prediction[~prediction.business_id.isn(rated_bus)]
            
        self.recommendation = self.recommendation.merge(prediction, on='business_id', how='inner')
        self.recommendation = self.recommendation.sort_values('stars_pred', ascending=False).reset_index(drop=True)
        self.display_columns.insert(0, 'stars_pred')
        self.display()
        
        return self.recommendation

In [28]:
%%time

# Instantiate the object
results = Recommender_Engine();

# Test case 1: Display results
print('Test case 1: *****------------*****\n');
results.display();

# Test case 1: No user_id input
print('Test case 1: *****------------*****\n');
results.collaborative_filtering();


Test case 1: *****------------*****

Below is the list of the top 10 recommended restaurants for you: 
                          name                           address        city  \
29761          Little Miss BBQ              4301 E University Dr     Phoenix   
2648              Brew Tea Bar      7380 S Rainbow Blvd, Ste 101   Las Vegas   
33734          Cocina Madrigal                    4044 S 16th St     Phoenix   
35172  Green Corner Restaurant        1038 W Southern Ave, Ste 1        Mesa   
3590            Worth Takeaway                     218 W Main St        Mesa   
9839            Zenaida's Cafe      3430 E Tropicana Ave, Ste 32   Las Vegas   
34397          Kodo Sushi Sake  15040 N Northsight Blvd, Ste 104  Scottsdale   
21628  Bajamar Seafood & Tacos             1615 S Las Vegas Blvd   Las Vegas   
11825                   Karved              3957 S Maryland Pkwy   Las Vegas   
34603    Not Your Typical Deli    1166 South Gilbert Rd, Ste 101     Gilbert   

      state attr

TypeError: object of type 'NoneType' has no len()